# Final

In [33]:
import os
import pandas as pd
import dicom2nifti
import dicom2nifti.settings
import time

# Optional: speed up conversion and allow for some non-standard DICOMs
dicom2nifti.settings.disable_validate_orthogonal()
dicom2nifti.settings.disable_validate_slice_increment()
dicom2nifti.settings.disable_validate_slicecount()

csv_path = 'train_with_folds.csv'
dicom_root = 'data/series'
nifti_root = 'data/niftiData'

os.makedirs(nifti_root, exist_ok=True)

df = pd.read_csv(csv_path)
df["ID"] = [str(i).zfill(4) for i in range(len(df))]

# Track original and new NIfTI file names and count
original_nifti_files = []
renamed_nifti_files = []
nifti_file_counts = []

for idx, row in df.iterrows():
    series_uid = str(row['SeriesInstanceUID'])
    modality = row['Modality']
    dicom_dir = os.path.join(dicom_root, series_uid)
    id_str = str(row['ID']) if 'ID' in row else str(idx).zfill(4)
    originals = []
    renamed = []
    if not os.path.isdir(dicom_dir):
        print(f"Warning: DICOM directory not found for {series_uid}")
        original_nifti_files.append('')
        renamed_nifti_files.append('')
        nifti_file_counts.append(0)
        continue
    before_files = set(os.listdir(nifti_root))
    try:
        print(f"Converting {series_uid} ({modality}) ...")
        dicom2nifti.convert_directory(dicom_dir, nifti_root, compression=True, reorient=True)
        time.sleep(2)  # Add 2 seconds delay
        after_files = set(os.listdir(nifti_root))
        new_files = after_files - before_files
        nii_files = [f for f in new_files if f.endswith('.nii.gz')]
        for i, nii_file in enumerate(nii_files):
            original_path = os.path.join(nifti_root, nii_file)
            new_nifti_name = f"{id_str}_{str(i).zfill(4)}.nii.gz"
            new_nifti_path = os.path.join(nifti_root, new_nifti_name)
            os.rename(original_path, new_nifti_path)
            originals.append(nii_file)
            renamed.append(new_nifti_name)
            print(f"   - Renamed {nii_file} -> {new_nifti_name}")
        original_nifti_files.append(';'.join(originals))
        renamed_nifti_files.append(';'.join(renamed))
        nifti_file_counts.append(len(nii_files))
        if not nii_files:
            print(f" > No NIfTI file created for {series_uid}")
    except Exception as e:
        print(f" > Failed to convert {series_uid}: {e}")
        original_nifti_files.append('')
        renamed_nifti_files.append('')
        nifti_file_counts.append(0)

# Add new columns to DataFrame and save
df['original_nifti_file'] = original_nifti_files
df['renamed_nifti_file'] = renamed_nifti_files
df['nifti_file_count'] = nifti_file_counts
df.to_csv('train_with_nifti.csv', index=False)
print('Updated CSV with original, renamed NIfTI file names, and file count saved as train_with_nifti.csv')

Converting 1.2.826.0.1.3680043.8.498.10004044428023505108375152878107656647 (MRA) ...
   - Renamed 1_ax_cow.nii.gz -> 0000_0000.nii.gz
Converting 1.2.826.0.1.3680043.8.498.10004684224894397679901841656954650085 (MRA) ...
   - Renamed 4_3d_tof_brain.nii.gz -> 0001_0000.nii.gz
Converting 1.2.826.0.1.3680043.8.498.10005158603912009425635473100344077317 (CTA) ...
   - Renamed 701_aneurysm.nii.gz -> 0002_0000.nii.gz
Converting 1.2.826.0.1.3680043.8.498.10009383108068795488741533244914370182 (MRA) ...
   - Renamed 11001_ax_3d_tof_whole_head.nii.gz -> 0003_0000.nii.gz
Converting 1.2.826.0.1.3680043.8.498.10012790035410518400400834395242853657 (MRA) ...
   - Renamed 601_mra.nii.gz -> 0004_0000.nii.gz
Converting 1.2.826.0.1.3680043.8.498.10014757658335054766479957992112625961 (CTA) ...
   - Renamed 501_cta_idose_3.nii.gz -> 0005_0000.nii.gz
Converting 1.2.826.0.1.3680043.8.498.10021411248005513321236647460239137906 (CTA) ...
   - Renamed 5_de_carotidangio__10__bv40__3__f_07.nii.gz -> 0006_0000.

Image orientations not consistent through all slices
---------------------------------------------------------
[1. 0. 0.] [1. 0. 0.]
[ 0.  0. -1.] [0. 1. 0.]
---------------------------------------------------------


   - Renamed 8_ax_t2_prop.nii.gz -> 1223_0000.nii.gz
Converting 1.2.826.0.1.3680043.8.498.13363897289235267814753067983525010231 (CTA) ...


Traceback (most recent call last):
  File "/home/jma/miniconda3/envs/ssl3d/lib/python3.10/site-packages/dicom2nifti/convert_dir.py", line 86, in convert_directory
    convert_dicom.dicom_array_to_nifti(dicom_input, nifti_file, reorient)
  File "/home/jma/miniconda3/envs/ssl3d/lib/python3.10/site-packages/dicom2nifti/convert_dicom.py", line 119, in dicom_array_to_nifti
    results = convert_generic.dicom_to_nifti(dicom_list, output_file)
  File "/home/jma/miniconda3/envs/ssl3d/lib/python3.10/site-packages/dicom2nifti/convert_generic.py", line 241, in dicom_to_nifti
    common.validate_orientation(dicom_input)
  File "/home/jma/miniconda3/envs/ssl3d/lib/python3.10/site-packages/dicom2nifti/common.py", line 1082, in validate_orientation
    raise ConversionValidationError('IMAGE_ORIENTATION_INCONSISTENT')
dicom2nifti.exceptions.ConversionValidationError: IMAGE_ORIENTATION_INCONSISTENT


 > No NIfTI file created for 1.2.826.0.1.3680043.8.498.13363897289235267814753067983525010231
Converting 1.2.826.0.1.3680043.8.498.13366884328602627495988583245074587769 (MRI T2) ...
   - Renamed 9.nii.gz -> 1225_0000.nii.gz
Converting 1.2.826.0.1.3680043.8.498.13368526455721722676241649119183398112 (MRA) ...
   - Renamed 601_mra.nii.gz -> 1226_0000.nii.gz
Converting 1.2.826.0.1.3680043.8.498.13377393892022203175801708136169006679 (CTA) ...
   - Renamed 8.nii.gz -> 1227_0000.nii.gz
Converting 1.2.826.0.1.3680043.8.498.13378390001124190052070266332960118576 (CTA) ...
   - Renamed 5_de_carotidangio_ax_soft_thin__f_07.nii.gz -> 1228_0000.nii.gz
Converting 1.2.826.0.1.3680043.8.498.13381897809166043991522637958719774589 (MRI T1post) ...
   - Renamed 10_cor_3dspgr_c.nii.gz -> 1229_0000.nii.gz
Converting 1.2.826.0.1.3680043.8.498.13383999413248798547416112334207116647 (MRI T1post) ...
   - Renamed 30_t1fs_3d_ax_whole_brain_gd.nii.gz -> 1230_0000.nii.gz
Converting 1.2.826.0.1.3680043.8.498.13

...

 > No NIfTI file created for 1.2.826.0.1.3680043.8.498.85668572260836671456680878948105863299
Converting 1.2.826.0.1.3680043.8.498.85694228896758469614431673786651945288 (MRI T2) ...
   - Renamed 701_t2_tse.nii.gz -> 3801_0000.nii.gz
Converting 1.2.826.0.1.3680043.8.498.85709849872024108265120796348331660195 (MRA) ...
   - Renamed 7_tof_3d_5x_multi-slab_384.nii.gz -> 3802_0000.nii.gz
Converting 1.2.826.0.1.3680043.8.498.85722501618335637418238816805460223934 (CTA) ...
   - Renamed 8_deidentified.nii.gz -> 3803_0000.nii.gz
Converting 1.2.826.0.1.3680043.8.498.85733266223939459614072318174725294585 (MRI T2) ...
   - Renamed 3_sag_t2.nii.gz -> 3804_0000.nii.gz
Converting 1.2.826.0.1.3680043.8.498.85743725023912475256847388501654113280 (CTA) ...
   - Renamed 4.nii.gz -> 3805_0000.nii.gz
Converting 1.2.826.0.1.3680043.8.498.85860352914625127150514616466982312044 (MRI T2) ...
   - Renamed 9_coronal_t2.nii.gz -> 3806_0000.nii.gz
Converting 1.2.826.0.1.3680043.8.498.859003947476941807923203153

Image orientations not consistent through all slices
---------------------------------------------------------
[1. 0. 0.] [0. 1. 0.]
[0. 1. 0.] [ 0.  0. -1.]
---------------------------------------------------------


   - Renamed 14001_tof_cs_acc120_large.nii.gz -> 3867_0000.nii.gz
Converting 1.2.826.0.1.3680043.8.498.87480891990277582946346790136781912242 (CTA) ...


Traceback (most recent call last):
  File "/home/jma/miniconda3/envs/ssl3d/lib/python3.10/site-packages/dicom2nifti/convert_dir.py", line 86, in convert_directory
    convert_dicom.dicom_array_to_nifti(dicom_input, nifti_file, reorient)
  File "/home/jma/miniconda3/envs/ssl3d/lib/python3.10/site-packages/dicom2nifti/convert_dicom.py", line 119, in dicom_array_to_nifti
    results = convert_generic.dicom_to_nifti(dicom_list, output_file)
  File "/home/jma/miniconda3/envs/ssl3d/lib/python3.10/site-packages/dicom2nifti/convert_generic.py", line 241, in dicom_to_nifti
    common.validate_orientation(dicom_input)
  File "/home/jma/miniconda3/envs/ssl3d/lib/python3.10/site-packages/dicom2nifti/common.py", line 1082, in validate_orientation
    raise ConversionValidationError('IMAGE_ORIENTATION_INCONSISTENT')
dicom2nifti.exceptions.ConversionValidationError: IMAGE_ORIENTATION_INCONSISTENT


 > No NIfTI file created for 1.2.826.0.1.3680043.8.498.87480891990277582946346790136781912242
Converting 1.2.826.0.1.3680043.8.498.87487378040032353374594537966032287706 (MRI T2) ...
   - Renamed 5_sag_t2.nii.gz -> 3869_0000.nii.gz
Converting 1.2.826.0.1.3680043.8.498.87547461665007390981451704461752148942 (MRA) ...
   - Renamed 18_mra__brain_3d_tof.nii.gz -> 3870_0000.nii.gz
Converting 1.2.826.0.1.3680043.8.498.87566514668273241564832485161915016907 (CTA) ...
   - Renamed 1124_aneurysm.nii.gz -> 3871_0000.nii.gz
Converting 1.2.826.0.1.3680043.8.498.87571543766634562655926322252071837705 (CTA) ...
   - Renamed 4.nii.gz -> 3872_0000.nii.gz
Converting 1.2.826.0.1.3680043.8.498.87596156951369172427618929370254044602 (CTA) ...
   - Renamed 502_angio__idose_3.nii.gz -> 3873_0000.nii.gz
Converting 1.2.826.0.1.3680043.8.498.87684836417769094239791074334923837991 (CTA) ...
   - Renamed 5_cta_stroke.nii.gz -> 3874_0000.nii.gz
Converting 1.2.826.0.1.3680043.8.498.87712930041363056817479223886738

In [ ]:
df[df['nifti_file_count'] == 0]

,SeriesInstanceUID,PatientAge,PatientSex,Modality,Left Infraclinoid Internal Carotid Artery,Right Infraclinoid Internal Carotid Artery,Left Supraclinoid Internal Carotid Artery,Right Supraclinoid Internal Carotid Artery,Left Middle Cerebral Artery,Right Middle Cerebral Artery,...,Left Posterior Communicating Artery,Right Posterior Communicating Artery,Basilar Tip,Other Posterior Circulation,Aneurysm Present,fold,ID,original_nifti_file,renamed_nifti_file,nifti_file_count
1224,1.2.826.0.1.3680043.8.498.13363897289235267814...,65,Male,CTA,0,0,0,0,0,0,...,0,0,0,0,0,2,1224,,,0
1256,1.2.826.0.1.3680043.8.498.13869493696081683922...,47,Female,CTA,0,0,0,0,0,0,...,0,0,0,0,0,2,1256,,,0
1266,1.2.826.0.1.3680043.8.498.14095224548874666070...,35,Female,CTA,0,0,0,0,0,0,...,0,1,0,0,1,1,1266,,,0
1392,1.2.826.0.1.3680043.8.498.17677548211553545296...,57,Male,CTA,0,0,0,0,0,0,...,0,0,0,0,1,2,1392,,,0
1599,1.2.826.0.1.3680043.8.498.23745303490352683267...,33,Female,CTA,0,0,0,0,0,0,...,1,0,0,0,1,2,1599,,,0
2168,1.2.826.0.1.3680043.8.498.39319917305377952606...,53,Female,CTA,0,0,0,0,1,0,...,0,0,0,0,1,3,2168,,,0
2699,1.2.826.0.1.3680043.8.498.54505273219907798567...,60,Female,CTA,0,0,0,0,0,1,...,0,0,0,0,1,4,2699,,,0
2714,1.2.826.0.1.3680043.8.498.54945937547396809705...,41,Female,CTA,0,0,1,0,0,0,...,0,0,0,1,1,3,2714,,,0
2726,1.2.826.0.1.3680043.8.498.55190777977920349777...,62,Female,CTA,0,0,0,1,0,0,...,0,0,0,0,1,0,2726,,,0
2828,1.2.826.0.1.3680043.8.498.57801812352381826359...,70,Male,CTA,0,0,0,0,0,0,...,0,0,0,0,0,2,2828,,,0


In [2]:
import pandas as pd

# Load original CSV
df = pd.read_csv("./train_with_nifti.csv")

# Keep only rows where nifti_file_count != 0
df_remaining = df[df["nifti_file_count"] != 0]

# Save to new CSV
df_remaining.to_csv("./train_with_nifti_remaining.csv", index=False)


# Extra

In [48]:
df[df['nifti_file_count'] == 0]['SeriesInstanceUID'].iloc[0]

'1.2.826.0.1.3680043.8.498.13363897289235267814753067983525010231'

In [ ]:
1.2.826.0.1.3680043.8.498.10004044428023505108375152878107656647 (MRA)
1.2.826.0.1.3680043.8.498.10004044428023505108375152878107656647

In [ ]:
### concurrent.futures.ProcessPoolExecutor ###
##############################################


import os
import pandas as pd
import dicom2nifti
import dicom2nifti.settings
import time
from concurrent.futures import ProcessPoolExecutor, as_completed
#
# dicom2nifti.settings.disable_validate_orthogonal()
dicom2nifti.settings.disable_validate_slice_increment()
dicom2nifti.settings.disable_validate_slicecount()

csv_path = 'train_with_folds.csv'
dicom_root = 'data/series'
nifti_root = 'data/niftiData2'
os.makedirs(nifti_root, exist_ok=True)

df = pd.read_csv(csv_path)
df["ID"] = [str(i).zfill(4) for i in range(len(df))]

def convert_row(row):
    import os, time
    series_uid = str(row['SeriesInstanceUID'])
    modality = row['Modality']
    dicom_dir = os.path.join(dicom_root, series_uid)
    id_str = str(row['ID'])
    originals = []
    renamed = []
    if not os.path.isdir(dicom_dir):
        return '', '', 0
    before_files = set(os.listdir(nifti_root))
    try:
        dicom2nifti.convert_directory(dicom_dir, nifti_root, compression=True, reorient=True)
        time.sleep(2)
        after_files = set(os.listdir(nifti_root))
        new_files = after_files - before_files
        nii_files = [f for f in new_files if f.endswith('.nii.gz')]
        for i, nii_file in enumerate(nii_files):
            original_path = os.path.join(nifti_root, nii_file)
            new_nifti_name = f"{id_str}_{str(i).zfill(4)}.nii.gz"
            new_nifti_path = os.path.join(nifti_root, new_nifti_name)
            os.rename(original_path, new_nifti_path)
            originals.append(nii_file)
            renamed.append(new_nifti_name)
        return ';'.join(originals), ';'.join(renamed), len(nii_files)
    except Exception as e:
        return '', '', 0

results = []
with ProcessPoolExecutor() as executor:
    futures = [executor.submit(convert_row, row) for _, row in df.iterrows()]
    for future in as_completed(futures):
        results.append(future.result())

# The results may be out of order, so sort by DataFrame index
results = [r for _, r in sorted(zip(df.index, results))]

df['original_nifti_file'] = [r[0] for r in results]
df['renamed_nifti_file'] = [r[1] for r in results]
df['nifti_file_count'] = [r[2] for r in results]
df.to_csv('train_with_nifti.csv', index=False)
print('Updated CSV with original, renamed NIfTI file names, and file count saved as train_with_nifti.csv')

In [ ]:
concurrent.futures.ProcessPoolExecutor

In [1]:
import os
import pandas as pd
csv_path = 'train_with_folds.csv'
dicom_root = 'data/series'
nifti_root = 'data/niftiData'

os.makedirs(nifti_root, exist_ok=True)

df = pd.read_csv(csv_path)
df["ID"] = [str(i).zfill(4) for i in range(len(df))]

In [ ]:
'data/series/1.2.826.0.1.3680043.8.498.10004044428023505108375152878107656647/1.2.826.0.1.3680043.8.498.97566471606280816297813988322177744412.dcm'
'data/series/1.2.826.0.1.3680043.8.498.10004684224894397679901841656954650085/1.2.826.0.1.3680043.8.498.35006729036793388746516667629927606398.dcm'

In [49]:
import os
# series_uid = "1.2.826.0.1.3680043.8.498.10004044428023505108375152878107656647"
# series_uid = "1.2.826.0.1.3680043.8.498.10004684224894397679901841656954650085"
os.listdir('data/series/1.2.826.0.1.3680043.8.498.13363897289235267814753067983525010231')
# ('data/series/1.2.826.0.1.3680043.8.498.10004684224894397679901841656954650085/1.2.826.0.1.3680043.8.498.35006729036793388746516667629927606398.dcm')
# ('1.2.826.0.1.3680043.8.498.35006729036793388746516667629927606398.dcm/1.2.826.0.1.3680043.8.498.35006729036793388746516667629927606398.dcm')

['1.2.826.0.1.3680043.8.498.12439578985238953344185113851909692223.dcm',
 '1.2.826.0.1.3680043.8.498.88384756855810033110095893833046300159.dcm',
 '1.2.826.0.1.3680043.8.498.61924536332585129747001404995897680043.dcm',
 '1.2.826.0.1.3680043.8.498.10242158849246451391735002669692489877.dcm',
 '1.2.826.0.1.3680043.8.498.58800750779616072979193599033977841161.dcm',
 '1.2.826.0.1.3680043.8.498.97759157189239598515275271168026399924.dcm',
 '1.2.826.0.1.3680043.8.498.34523178643009035971690185731933959852.dcm',
 '1.2.826.0.1.3680043.8.498.99744767692790343461193764027435942141.dcm',
 '1.2.826.0.1.3680043.8.498.66177756013310104927614044109289308855.dcm',
 '1.2.826.0.1.3680043.8.498.14243617751139681057952330522477033299.dcm',
 '1.2.826.0.1.3680043.8.498.89314965599028271620450678048847603552.dcm',
 '1.2.826.0.1.3680043.8.498.13082049894064227421271667363096636847.dcm',
 '1.2.826.0.1.3680043.8.498.11700329604130848819217209568801507884.dcm',
 '1.2.826.0.1.3680043.8.498.30572563759074438036979

In [ ]:
Converting 1.2.826.0.1.3680043.8.498.10004044428023505108375152878107656647 (MRA) ...
   - Renamed 1_ax_cow.nii.gz -> 0000_0000.nii.gz
Converting 1.2.826.0.1.3680043.8.498.10004684224894397679901841656954650085 (MRA) ...
   - Renamed 1_ax_cow.nii.gz -> 0000_0000.nii.gz
Converting 1.2.826.0.1.3680043.8.498.10004684224894397679901841656954650085 (MRA) ...
   - Renamed 4_3d_tof_brain.nii.gz -> 0001_0000.nii.gz
Converting 1.2.826.0.1.3680043.8.498.10005158603912009425635473100344077317 (CTA) ...
   - Renamed 4_3d_tof_brain.nii.gz -> 0001_0000.nii.gz
Converting 1.2.826.0.1.3680043.8.498.10005158603912009425635473100344077317 (CTA) ...
   - Renamed 701_aneurysm.nii.gz -> 0002_0000.nii.gz

In [ ]:
'data/series/1.2.826.0.1.3680043.8.498.10004044428023505108375152878107656647/1.2.826.0.1.3680043.8.498.97566471606280816297813988322177744412.dcm'
'data/series/1.2.826.0.1.3680043.8.498.10004684224894397679901841656954650085/1.2.826.0.1.3680043.8.498.35006729036793388746516667629927606398.dcm'

In [ ]:
(0010,0010) Patient's Name                      PN: 'e497f36e-4b5'
(0010,0020) Patient ID                          LO: 'e497f36e-4b5'
(0010,0040) Patient's Sex                       CS: 'F'
(0010,1010) Patient's Age                       AS: '075Y'
(0010,1030) Patient's Weight                    DS: '45'
e497f36e-4b5
20280403



In [15]:
import pydicom

# Path to your DICOM file
# dicom_file = 'data/series/1.2.826.0.1.3680043.8.498.10004684224894397679901841656954650085/1.2.826.0.1.3680043.8.498.35006729036793388746516667629927606398.dcm'
dicom_file='data/series/1.2.826.0.1.3680043.8.498.10004044428023505108375152878107656647/1.2.826.0.1.3680043.8.498.97566471606280816297813988322177744412.dcm'
# Read the DICOM file
ds = pydicom.dcmread(dicom_file)



# To access specific metadata, for example:
print(ds.PatientID)
print(ds.StudyDate)
print(ds.Modality)


print(ds)

76dc135c-f3d
20230531
MR
Dataset.file_meta -------------------------------
(0002,0000) File Meta Information Group Length  UL: 208
(0002,0001) File Meta Information Version       OB: b'\x00\x01'
(0002,0002) Media Storage SOP Class UID         UI: MR Image Storage
(0002,0003) Media Storage SOP Instance UID      UI: 1.2.826.0.1.3680043.8.498.97700683413986996884378200143631889592
(0002,0010) Transfer Syntax UID                 UI: RLE Lossless
(0002,0012) Implementation Class UID            UI: 1.2.276.0.7230010.3.0.3.6.8
(0002,0013) Implementation Version Name         SH: 'OFFIS_DCMTK_368'
-------------------------------------------------
(0008,0005) Specific Character Set              CS: 'ISO_IR 100'
(0008,0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'M', 'ND', 'NORM']
(0008,0016) SOP Class UID                       UI: MR Image Storage
(0008,0018) SOP Instance UID                    UI: 1.2.826.0.1.3680043.8.498.97566471606280816297813988322177744412
(0008,00

In [9]:
df[df['SeriesInstanceUID']=='1.2.826.0.1.3680043.8.498.10004684224894397679901841656954650085']

,SeriesInstanceUID,PatientAge,PatientSex,Modality,Left Infraclinoid Internal Carotid Artery,Right Infraclinoid Internal Carotid Artery,Left Supraclinoid Internal Carotid Artery,Right Supraclinoid Internal Carotid Artery,Left Middle Cerebral Artery,Right Middle Cerebral Artery,Anterior Communicating Artery,Left Anterior Cerebral Artery,Right Anterior Cerebral Artery,Left Posterior Communicating Artery,Right Posterior Communicating Artery,Basilar Tip,Other Posterior Circulation,Aneurysm Present
1,1.2.826.0.1.3680043.8.498.10004684224894397679...,76,Female,MRA,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
type(df['ID'][0])

str

In [62]:
import os
import pandas as pd
import dicom2nifti
import dicom2nifti.settings

# Optional: speed up conversion and allow for some non-standard DICOMs

dicom2nifti.settings.disable_validate_slicecount()   # turn off slice count check
dicom2nifti.settings.disable_validate_orientation()  # disable orientation check
dicom2nifti.settings.disable_validate_orthogonal()
dicom2nifti.settings.disable_validate_slice_increment()
dicom2nifti.settings.disable_validate_slicecount()

csv_path = 'train.csv'
dicom_root = 'data/series'
nifti_root = 'data/niftiData_extra'

os.makedirs(nifti_root, exist_ok=True)

df = pd.read_csv(csv_path)

# Select one participant by index (e.g., the first row)
row = df.iloc[1224]
series_uid = str(row['SeriesInstanceUID'])
modality = row['Modality']
dicom_dir = os.path.join(dicom_root, series_uid)
nifti_path = os.path.join(nifti_root, f"{series_uid}.nii.gz")

if not os.path.isdir(dicom_dir):
    print(f"Warning: DICOM directory not found for {series_uid}")
else:
    try:
        print(f"Converting {series_uid} ({modality}) ...")
        dicom2nifti.convert_directory(dicom_dir, nifti_root, compression=True, reorient=True)
        # Rename output to match {series_uid}.nii.gz
        for f in os.listdir(nifti_root):
            print (f)
            if f.startswith(series_uid) and f.endswith('.nii.gz') and f != f"{series_uid}.nii.gz":
                os.rename(os.path.join(nifti_root, f), nifti_path)
        print(f"Saved: {nifti_path}")
    except Exception as e:
        print(f"Failed to convert {series_uid}: {e}")

Converting 1.2.826.0.1.3680043.8.498.13363897289235267814753067983525010231 (CTA) ...


Missing slices (slice count mismatch between timepoint 0 and 1)
---------------------------------------------------------
(512, 512, 2)
(512, 512, 50)
---------------------------------------------------------


1.2.826.0.1.3680043.8.498.13363897289235267814753067983525010231
Saved: data/niftiData_extra/1.2.826.0.1.3680043.8.498.13363897289235267814753067983525010231.nii.gz


Traceback (most recent call last):
  File "/home/jma/miniconda3/envs/ssl3d/lib/python3.10/site-packages/dicom2nifti/convert_dir.py", line 86, in convert_directory
    convert_dicom.dicom_array_to_nifti(dicom_input, nifti_file, reorient)
  File "/home/jma/miniconda3/envs/ssl3d/lib/python3.10/site-packages/dicom2nifti/convert_dicom.py", line 119, in dicom_array_to_nifti
    results = convert_generic.dicom_to_nifti(dicom_list, output_file)
  File "/home/jma/miniconda3/envs/ssl3d/lib/python3.10/site-packages/dicom2nifti/convert_generic.py", line 252, in dicom_to_nifti
    return four_d_to_nifti(grouped_dicoms, output_file)
  File "/home/jma/miniconda3/envs/ssl3d/lib/python3.10/site-packages/dicom2nifti/convert_generic.py", line 82, in four_d_to_nifti
    full_block = _get_full_block(grouped_dicoms)
  File "/home/jma/miniconda3/envs/ssl3d/lib/python3.10/site-packages/dicom2nifti/convert_generic.py", line 127, in _get_full_block
    raise ConversionError("MISSING_DICOM_FILES")
dicom2nifti.ex

In [58]:
import os
import pandas as pd
import dicom2nifti
import dicom2nifti.settings

# Disable strict validation checks
dicom2nifti.settings.disable_validate_orientation()
dicom2nifti.settings.disable_validate_orthogonal()
dicom2nifti.settings.disable_validate_slice_increment()
dicom2nifti.settings.disable_validate_slicecount()

csv_path = 'train.csv'
dicom_root = 'data/series'         # leave as is
nifti_root = 'data/niftiData_extra'  # all NIfTI outputs go here

os.makedirs(nifti_root, exist_ok=True)

df = pd.read_csv(csv_path)

# Pick one row (e.g., 1224)
row = df.iloc[1224]
series_uid = str(row['SeriesInstanceUID'])
modality = row['Modality']

dicom_dir = os.path.join(dicom_root, series_uid)

# Create a dedicated subfolder for this series inside nifti_root
out_dir = os.path.join(nifti_root, series_uid)
os.makedirs(out_dir, exist_ok=True)

# Final NIfTI path
nifti_path = os.path.join(out_dir, f"{series_uid}.nii.gz")

if not os.path.isdir(dicom_dir):
    print(f"Warning: DICOM directory not found for {series_uid}")
else:
    try:
        print(f"Converting {series_uid} ({modality}) ...")
        dicom2nifti.convert_directory(dicom_dir, out_dir, compression=True, reorient=True)

        # Rename output to match exactly {series_uid}.nii.gz
        for f in os.listdir(out_dir):
            if f.endswith('.nii.gz') and f != f"{series_uid}.nii.gz":
                os.rename(os.path.join(out_dir, f), nifti_path)

        print(f"Saved: {nifti_path}")
    except Exception as e:
        print(f"Failed to convert {series_uid}: {e}")


Converting 1.2.826.0.1.3680043.8.498.13363897289235267814753067983525010231 (CTA) ...


Missing slices (slice count mismatch between timepoint 0 and 1)
---------------------------------------------------------
(512, 512, 2)
(512, 512, 50)
---------------------------------------------------------


Saved: data/niftiData_extra/1.2.826.0.1.3680043.8.498.13363897289235267814753067983525010231/1.2.826.0.1.3680043.8.498.13363897289235267814753067983525010231.nii.gz


Traceback (most recent call last):
  File "/home/jma/miniconda3/envs/ssl3d/lib/python3.10/site-packages/dicom2nifti/convert_dir.py", line 86, in convert_directory
    convert_dicom.dicom_array_to_nifti(dicom_input, nifti_file, reorient)
  File "/home/jma/miniconda3/envs/ssl3d/lib/python3.10/site-packages/dicom2nifti/convert_dicom.py", line 119, in dicom_array_to_nifti
    results = convert_generic.dicom_to_nifti(dicom_list, output_file)
  File "/home/jma/miniconda3/envs/ssl3d/lib/python3.10/site-packages/dicom2nifti/convert_generic.py", line 252, in dicom_to_nifti
    return four_d_to_nifti(grouped_dicoms, output_file)
  File "/home/jma/miniconda3/envs/ssl3d/lib/python3.10/site-packages/dicom2nifti/convert_generic.py", line 82, in four_d_to_nifti
    full_block = _get_full_block(grouped_dicoms)
  File "/home/jma/miniconda3/envs/ssl3d/lib/python3.10/site-packages/dicom2nifti/convert_generic.py", line 127, in _get_full_block
    raise ConversionError("MISSING_DICOM_FILES")
dicom2nifti.ex

In [59]:
import os
import shutil
import pandas as pd
import pydicom
import dicom2nifti
import dicom2nifti.settings

# Disable strict validation checks
dicom2nifti.settings.disable_validate_orientation()
dicom2nifti.settings.disable_validate_orthogonal()
dicom2nifti.settings.disable_validate_slice_increment()
dicom2nifti.settings.disable_validate_slicecount()

csv_path = 'train.csv'
dicom_root = 'data/series'           # leave as is (raw DICOMs)
dicom_split_root = 'data/series_split'  # split dicoms by SeriesInstanceUID
nifti_root = 'data/niftiData_extra'  # all NIfTI outputs go here

os.makedirs(dicom_split_root, exist_ok=True)
os.makedirs(nifti_root, exist_ok=True)

# --- STEP 1: Split dicoms into per-series folders ---
for fname in os.listdir(dicom_root):
    fpath = os.path.join(dicom_root, fname)
    if not os.path.isfile(fpath):
        continue
    try:
        dcm = pydicom.dcmread(fpath, stop_before_pixels=True)
        series_uid = dcm.SeriesInstanceUID
        out_dir = os.path.join(dicom_split_root, series_uid)
        os.makedirs(out_dir, exist_ok=True)
        shutil.copy(fpath, os.path.join(out_dir, fname))
    except Exception as e:
        print(f"Skipping {fname}: {e}")

# # --- STEP 2: Pick one row from train.csv and convert that series ---
# df = pd.read_csv(csv_path)

# # Example: row 1224
# row = df.iloc[1224]
# series_uid = str(row['SeriesInstanceUID'])
# modality = row['Modality']

# dicom_dir = os.path.join(dicom_split_root, series_uid)  # now use split dir
# out_dir = os.path.join(nifti_root, series_uid)          # nifti output folder
# os.makedirs(out_dir, exist_ok=True)

# # Final NIfTI path
# nifti_path = os.path.join(out_dir, f"{series_uid}.nii.gz")

# if not os.path.isdir(dicom_dir):
#     print(f"Warning: DICOM directory not found for {series_uid}")
# else:
#     try:
#         print(f"Converting {series_uid} ({modality}) ...")
#         dicom2nifti.convert_directory(dicom_dir, out_dir, compression=True, reorient=True)

#         # Rename output to match exactly {series_uid}.nii.gz
#         for f in os.listdir(out_dir):
#             if f.endswith('.nii.gz') and f != f"{series_uid}.nii.gz":
#                 os.rename(os.path.join(out_dir, f), nifti_path)

#         print(f"Saved: {nifti_path}")
#     except Exception as e:
#         print(f"Failed to convert {series_uid}: {e}")


In [ ]:
dicom_root = 'data/series'

In [60]:
import os
import pydicom
import shutil

src = 'data/series/1.2.826.0.1.3680043.8.498.13363897289235267814753067983525010231'
dst = "dicoms_split/"

os.makedirs(dst, exist_ok=True)

for f in os.listdir(src):
    fpath = os.path.join(src, f)
    if not os.path.isfile(fpath):
        continue
    try:
        dcm = pydicom.dcmread(fpath, stop_before_pixels=True)  # faster, metadata only
        series_uid = dcm.SeriesInstanceUID
        out_dir = os.path.join(dst, series_uid)
        os.makedirs(out_dir, exist_ok=True)
        shutil.copy(fpath, os.path.join(out_dir, f))
    except Exception as e:
        print(f"Skipping {f}: {e}")


In [61]:
import os
import dicom2nifti
import dicom2nifti.settings

# Disable strict validation (since you had missing slice/orientation issues)
dicom2nifti.settings.disable_validate_orientation()
dicom2nifti.settings.disable_validate_orthogonal()
dicom2nifti.settings.disable_validate_slice_increment()
dicom2nifti.settings.disable_validate_slicecount()

series_uid = "1.2.826.0.1.3680043.8.498.13363897289235267814753067983525010231"
dicom_dir = os.path.join("dicoms_split", series_uid)
nifti_out_dir = os.path.join("nifti_out", series_uid)
os.makedirs(nifti_out_dir, exist_ok=True)

nifti_path = os.path.join(nifti_out_dir, f"{series_uid}.nii.gz")

try:
    print(f"Converting {series_uid} ...")
    dicom2nifti.convert_directory(dicom_dir, nifti_out_dir, compression=True, reorient=True)

    # Rename if needed
    for f in os.listdir(nifti_out_dir):
        if f.endswith(".nii.gz") and f != f"{series_uid}.nii.gz":
            os.rename(os.path.join(nifti_out_dir, f), nifti_path)

    print(f"Saved: {nifti_path}")
except Exception as e:
    print(f"Failed: {e}")


Converting 1.2.826.0.1.3680043.8.498.13363897289235267814753067983525010231 ...


Missing slices (slice count mismatch between timepoint 0 and 1)
---------------------------------------------------------
(512, 512, 2)
(512, 512, 50)
---------------------------------------------------------


Saved: nifti_out/1.2.826.0.1.3680043.8.498.13363897289235267814753067983525010231/1.2.826.0.1.3680043.8.498.13363897289235267814753067983525010231.nii.gz


Traceback (most recent call last):
  File "/home/jma/miniconda3/envs/ssl3d/lib/python3.10/site-packages/dicom2nifti/convert_dir.py", line 86, in convert_directory
    convert_dicom.dicom_array_to_nifti(dicom_input, nifti_file, reorient)
  File "/home/jma/miniconda3/envs/ssl3d/lib/python3.10/site-packages/dicom2nifti/convert_dicom.py", line 119, in dicom_array_to_nifti
    results = convert_generic.dicom_to_nifti(dicom_list, output_file)
  File "/home/jma/miniconda3/envs/ssl3d/lib/python3.10/site-packages/dicom2nifti/convert_generic.py", line 252, in dicom_to_nifti
    return four_d_to_nifti(grouped_dicoms, output_file)
  File "/home/jma/miniconda3/envs/ssl3d/lib/python3.10/site-packages/dicom2nifti/convert_generic.py", line 82, in four_d_to_nifti
    full_block = _get_full_block(grouped_dicoms)
  File "/home/jma/miniconda3/envs/ssl3d/lib/python3.10/site-packages/dicom2nifti/convert_generic.py", line 127, in _get_full_block
    raise ConversionError("MISSING_DICOM_FILES")
dicom2nifti.ex

In [52]:
df[df['nifti_file_count'] == 0]#['SeriesInstanceUID']

,SeriesInstanceUID,PatientAge,PatientSex,Modality,Left Infraclinoid Internal Carotid Artery,Right Infraclinoid Internal Carotid Artery,Left Supraclinoid Internal Carotid Artery,Right Supraclinoid Internal Carotid Artery,Left Middle Cerebral Artery,Right Middle Cerebral Artery,...,Left Posterior Communicating Artery,Right Posterior Communicating Artery,Basilar Tip,Other Posterior Circulation,Aneurysm Present,fold,ID,original_nifti_file,renamed_nifti_file,nifti_file_count
1224,1.2.826.0.1.3680043.8.498.13363897289235267814...,65,Male,CTA,0,0,0,0,0,0,...,0,0,0,0,0,2,1224,,,0
1256,1.2.826.0.1.3680043.8.498.13869493696081683922...,47,Female,CTA,0,0,0,0,0,0,...,0,0,0,0,0,2,1256,,,0
1266,1.2.826.0.1.3680043.8.498.14095224548874666070...,35,Female,CTA,0,0,0,0,0,0,...,0,1,0,0,1,1,1266,,,0
1392,1.2.826.0.1.3680043.8.498.17677548211553545296...,57,Male,CTA,0,0,0,0,0,0,...,0,0,0,0,1,2,1392,,,0
1599,1.2.826.0.1.3680043.8.498.23745303490352683267...,33,Female,CTA,0,0,0,0,0,0,...,1,0,0,0,1,2,1599,,,0
2168,1.2.826.0.1.3680043.8.498.39319917305377952606...,53,Female,CTA,0,0,0,0,1,0,...,0,0,0,0,1,3,2168,,,0
2699,1.2.826.0.1.3680043.8.498.54505273219907798567...,60,Female,CTA,0,0,0,0,0,1,...,0,0,0,0,1,4,2699,,,0
2714,1.2.826.0.1.3680043.8.498.54945937547396809705...,41,Female,CTA,0,0,1,0,0,0,...,0,0,0,1,1,3,2714,,,0
2726,1.2.826.0.1.3680043.8.498.55190777977920349777...,62,Female,CTA,0,0,0,1,0,0,...,0,0,0,0,1,0,2726,,,0
2828,1.2.826.0.1.3680043.8.498.57801812352381826359...,70,Male,CTA,0,0,0,0,0,0,...,0,0,0,0,0,2,2828,,,0


In [ ]:
df.loc[1224] 

SeriesInstanceUID                             1.2.826.0.1.3680043.8.498.13363897289235267814...
PatientAge                                                                                   65
PatientSex                                                                                 Male
Modality                                                                                    CTA
Left Infraclinoid Internal Carotid Artery                                                     0
Right Infraclinoid Internal Carotid Artery                                                    0
Left Supraclinoid Internal Carotid Artery                                                     0
Right Supraclinoid Internal Carotid Artery                                                    0
Left Middle Cerebral Artery                                                                   0
Right Middle Cerebral Artery                                                                  0
Anterior Communicating Artery           

In [ ]:
1.2.826.0.1.3680043.8.498.13363897289235267814753067983525010231


In [28]:
df.iloc[1]

SeriesInstanceUID                             1.2.826.0.1.3680043.8.498.10004684224894397679...
PatientAge                                                                                   76
PatientSex                                                                               Female
Modality                                                                                    MRA
Left Infraclinoid Internal Carotid Artery                                                     0
Right Infraclinoid Internal Carotid Artery                                                    0
Left Supraclinoid Internal Carotid Artery                                                     0
Right Supraclinoid Internal Carotid Artery                                                    0
Left Middle Cerebral Artery                                                                   0
Right Middle Cerebral Artery                                                                  0
Anterior Communicating Artery           

In [26]:
row = df[df['SeriesInstanceUID'] == series_uid].iloc[0]
row = df.iloc[0]
row

SeriesInstanceUID                             1.2.826.0.1.3680043.8.498.10004044428023505108...
PatientAge                                                                                   64
PatientSex                                                                               Female
Modality                                                                                    MRA
Left Infraclinoid Internal Carotid Artery                                                     0
Right Infraclinoid Internal Carotid Artery                                                    0
Left Supraclinoid Internal Carotid Artery                                                     0
Right Supraclinoid Internal Carotid Artery                                                    0
Left Middle Cerebral Artery                                                                   0
Right Middle Cerebral Artery                                                                  0
Anterior Communicating Artery           

In [22]:
series_uid

'1.2.826.0.1.3680043.8.498.10004044428023505108375152878107656647'

In [24]:
series_uid = "1.2.826.0.1.3680043.8.498.10012790035410518400400834395242853657"
series_uid

'1.2.826.0.1.3680043.8.498.10012790035410518400400834395242853657'

In [9]:
df = pd.read_csv(csv_path)



series_uid = "1.2.826.0.1.3680043.8.498.10004044428023505108375152878107656647"
row = df[df['SeriesInstanceUID'] == series_uid].iloc[0]

# Select one participant by index (e.g., the first row)
row = df.iloc[0]
series_uid = str(row['SeriesInstanceUID'])
series_uid 

'1.2.826.0.1.3680043.8.498.10004044428023505108375152878107656647'

In [7]:
dicom2nifti.convert_directory?


Signature:
dicom2nifti.convert_directory(
    dicom_directory,
    output_folder,
    compression=True,
    reorient=True,
)
Docstring:
This function will order all dicom files by series and order them one by one

:param compression: enable or disable gzip compression
:param reorient: reorient the dicoms according to LAS orientation
:param output_folder: folder to write the nifti files to
:param dicom_directory: directory with dicom files
File:      ~/miniconda3/envs/ssl3d/lib/python3.10/site-packages/dicom2nifti/convert_dir.py
Type:      function

In [4]:
df[df['SeriesInstanceUID'] == series_uid].iloc[0]

SeriesInstanceUID                             1.2.826.0.1.3680043.8.498.10004044428023505108...
PatientAge                                                                                   64
PatientSex                                                                               Female
Modality                                                                                    MRA
Left Infraclinoid Internal Carotid Artery                                                     0
Right Infraclinoid Internal Carotid Artery                                                    0
Left Supraclinoid Internal Carotid Artery                                                     0
Right Supraclinoid Internal Carotid Artery                                                    0
Left Middle Cerebral Artery                                                                   0
Right Middle Cerebral Artery                                                                  0
Anterior Communicating Artery           